In [532]:
# using Statistics
import Base
using StaticArrays;

In [503]:
struct Individual
    position
    cost
end

In [504]:
struct Output
    pop
    bestsol
    bestcost
end

In [505]:
struct Problem
    costfunc
    nvar
    varmin
    varmax
end

In [506]:
struct Parameter
    maxit
    npop
    beta
    pc
    gamma
    mu
    sugma
end

In [613]:
function exp_array(x)
    out = zeros(length(x))
    for i in 1:length(x)
        out[i] = exp(x[i])
    end
    return out
end

exp_array (generic function with 1 method)

In [702]:
function roulette_wheel_selection(p)
    c = cumsum(p, dims = 1)
    r = Base.sum(p)*rand()
    ind = 0
    for i in 1:length(c)
        if r <= c[i]
            ind = i
            break
        end
    end
    return ind
end

roulette_wheel_selection (generic function with 1 method)

In [1057]:
function  crossover(p1, p2, gamma=0.1)
    c1 = deepcopy(p1)
    c2 = deepcopy(p1)
    alpha = rand(length(c1.position))*(1+2*gamma).-gamma
    c1 = Individual(alpha.*p1.position + (1 .-alpha).*p2.position, c1.cost)
    c2 = Individual(alpha.*p2.position + (1 .-alpha).*p1.position, c2.cost)
    return c1, c2
end

crossover (generic function with 2 methods)

In [746]:
function mutate(x, mu, sugma)
    y = deepcopy(x)
    ind  = []
    for i in 1:length(x.position)
        if rand()<= mu
            append!(ind,i)
        end
    end
    y.position[ind] += sugma*rand(length(ind))
    return y
end

mutate (generic function with 1 method)

In [801]:
function max_v(array, max)
    for i in 1:length(array)
        if array[i] < max[i]
            array[i] = max[i]
        end
    end
    return array
end

max_v (generic function with 1 method)

In [802]:
function min_v(array, min)
    for i in 1:length(array)
        if array[i] > min[i]
            array[i] = min[i]
        end
    end
    return array
end

min_v (generic function with 1 method)

In [803]:
function apply_bound(x, varmin, varmax)
    x = Individual(max_v(x.position, varmin), x.cost)
    x = Individual(min_v(x.position, varmax), x.cost)
    return x
end

apply_bound (generic function with 1 method)

In [1071]:
function sortbycost(pop)
    for i in 1:length(pop)
        for j in 1:length(pop)
            if pop[i].cost < pop[j].cost
                tmp = pop[j]
                pop[j] = pop[i]
                pop[i] = tmp
            end
        end
    end
    return pop
end

sortbycost (generic function with 1 method)

In [1117]:
function run(problem, params)
    # Problem Information
    costfunc = problem.costfunc
    nvar = problem.nvar
    varmin = problem.varmin
    varmax = problem.varmax

    # Parameters
    maxit = params.maxit
    npop = params.npop
    beta = params.beta
    pc = params.pc
    nc = Int(floor(pc*npop/2)*2)
    gamma = params.gamma
    mu = params.mu
    sugma = params.sugma
    
    # Empty Individual Template
    empty_individual = Individual(Cvoid,Cvoid)
    bestsol = deepcopy(empty_individual)
    # bestsol.cost = Inf
    bestsol = Individual(bestsol.position, Inf)
    
    # Initialize Population
    pop=[]
    for i in 1:npop
       push!(pop,empty_individual)
    end
    for i in 1:npop
        pos = rand(nvar).*(varmax-varmin).+varmin
        costc = costfunc(pos)
        pop[i] = Individual(pos, costc)
        if pop[i].cost < bestsol.cost
            bestsol = deepcopy(pop[i])
        end
    end
    
    # Best Cost of Iterations 
    bestcost = zeros(maxit)
    
    # Main Loop
    for it in 1:maxit
        costs = []
        for i in 1:length(pop)
            append!(costs, pop[i].cost)
        end
        avg_cost = mean(costs)
        if avg_cost != 0
            costs = costs ./avg_cost
        end
        probs = exp_array(-beta.*costs)
        
        popc = []
        for _ in 1:floor(nc/2)
            # Perform Roulette Wheel Selection
            p1 = pop[roulette_wheel_selection(probs)]
            p2 = pop[roulette_wheel_selection(probs)]
            
            # Perform Crossover
            c1, c2 = crossover(p1, p2, gamma)
            
            # Perform Mutation
            c1 = mutate(c1, mu, sugma)
            c2 = mutate(c2, mu, sugma)
            
            # Apply Bounds
            c1 = apply_bound(c1, varmin, varmax)
            c2 = apply_bound(c2, varmin, varmax)
#             println("c1 = ",c1.position)
#             println("c2 = ",c2.position)
            
            # Evaluate First Offspring
            c1 = Individual(c1.position, costfunc(c1.position))
            if c1.cost < bestsol.cost
                bestsol = deepcopy(c1)
            end

            # Evaluate Second Offspring
            c2 = Individual(c2.position, costfunc(c2.position))
            if c2.cost < bestsol.cost
                bestsol = deepcopy(c2)
            end
            
            # Add Offsprings to popc
            push!(popc,c1)
            push!(popc,c2)
        end
                
        # Merge, Sort and Select 87
        for i in 1:length(popc)
            push!(pop,popc[i])
        end
        pop = sortbycost(pop)
        pop = deepcopy(pop[1:npop])

        # Store Best Cost
        bestcost[it] = bestsol.cost

        # Show Iteration Information
        println("Iteration "*string(it)*": Best Cost = "*string(bestcost[it]))
    end
    # Output
    # out = Output(pop,bestsol,bestcost)
    return bestsol
end

run (generic function with 1 method)

In [1114]:
function sphere(x)
    return Base.sum(x.^2)
end

sphere (generic function with 1 method)

In [1115]:
problem = Problem(sphere, 5, [-10, -10, -1, -5,  4],[ 10,  10,  1,  5, 10])
params  = Parameter(100, 50, 1, 1, 0.1, 0.01, 0.1)

Parameter(100, 50, 1, 1, 0.1, 0.01, 0.1)

In [1124]:
out = run(problem, params)

Iteration 1: Best Cost = 25.045836853443284
Iteration 2: Best Cost = 25.045836853443284
Iteration 3: Best Cost = 18.629461898575347
Iteration 4: Best Cost = 18.629461898575347
Iteration 5: Best Cost = 18.07909919773151
Iteration 6: Best Cost = 17.80656736823865
Iteration 7: Best Cost = 17.058058368177555
Iteration 8: Best Cost = 16.328009701190677
Iteration 9: Best Cost = 16.328009701190677
Iteration 10: Best Cost = 16.328009701190677
Iteration 11: Best Cost = 16.328009701190677
Iteration 12: Best Cost = 16.328009701190677
Iteration 13: Best Cost = 16.31276663746324
Iteration 14: Best Cost = 16.294048522608495
Iteration 15: Best Cost = 16.29314468628126
Iteration 16: Best Cost = 16.269735445600677
Iteration 17: Best Cost = 16.246712458329842
Iteration 18: Best Cost = 16.240419217225323
Iteration 19: Best Cost = 16.234063160293324
Iteration 20: Best Cost = 16.221557145544008
Iteration 21: Best Cost = 16.221557145544008
Iteration 22: Best Cost = 16.221557145544008
Iteration 23: Best Cost

Individual([0.022234397900655293, 0.06904390655228099, -0.00044208516722147765, 5.750383747505093e-5, 4.0249619211553265], 16.20558009497837)

In [1103]:
out.cost

17.058692677639076